# Datarock Code Challenge
## Step 1: Load and check data

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
# reset directory if required
# import os
# os.getcwd()
# os.chdir('C:\\Users\\ksil8584\\OneDrive - The University of Sydney (Staff)\\Documents\\GitHub\\coding-test\\')

In [3]:
data_file = './/data//data_for_distribution.csv'
raw_data = pd.read_csv(data_file)

In [4]:
# general stats for table
raw_data.describe()

,from,to,As,Pb,Fe,Mo,Cu,S,Zn
count,4771.000000,4771.000000,3268.000000,4756.000000,4709.000000,4741.000000,4746.000000,4761.000000,4762.000000
mean,750.379585,760.353574,19.730855,689.831232,49952.514598,9.991452,12.450601,9750.033213,59.389636
std,447.126995,447.114592,37.181529,1047.642566,21490.606419,87.098943,107.438873,15557.657335,120.489477
min,71.000000,81.000000,1.000000,1.600000,2080.000000,-999.000000,1.000000,26.000000,5.600000
25%,421.000000,431.000000,5.400000,132.200000,39260.000000,1.400000,3.000000,1338.000000,29.800000
50%,641.000000,651.000000,9.200000,396.700000,49020.000000,4.400000,4.600000,3636.000000,38.200000
75%,991.000000,1001.000000,20.000000,940.200000,58420.000000,17.400000,8.000000,10988.000000,52.600000
max,2201.000000,2211.000000,827.800000,29793.800000,397000.000000,1939.400000,6767.000000,217600.000000,3455.000000


In [5]:
#find nan
raw_data.isna().sum()

Unique_ID       0
holeid          0
from            0
to              0
As           1503
Au              6
Pb             15
Fe             62
Mo             30
Cu             25
S              10
Zn              9
Class           0
dtype: int64

In [6]:
# too many As values are missing this element will not be used in this analysis
# drop lines with missing values in other elements
data_p1 = raw_data.copy(deep=True)
data_p1 = data_p1.dropna(subset = ['Au', 'Pb', 'Fe','Mo','Cu','S','Zn'])

#check for remaining nan
data_p1.isna().sum()

Unique_ID       0
holeid          0
from            0
to              0
As           1456
Au              0
Pb              0
Fe              0
Mo              0
Cu              0
S               0
Zn              0
Class           0
dtype: int64

In [7]:
#Au is missing from the stats table above, indicating non-numeric data
# to confirm check data types
data_p1.dtypes

Unique_ID     object
holeid        object
from           int64
to           float64
As           float64
Au            object
Pb           float64
Fe           float64
Mo           float64
Cu           float64
S            float64
Zn           float64
Class         object
dtype: object

In [8]:
# Looking at the data, the problem is the <0.005 detection limit. For this analysis they will be changed to the limit
data_p1['Au'] = data_p1['Au'].replace(['<0.005'], ['0.005'])
data_p1['Au'] = data_p1['Au'].astype(float)

In [9]:
data_p1.describe()

,from,to,As,Au,Pb,Fe,Mo,Cu,S,Zn
count,4643.000000,4643.000000,3187.000000,4643.000000,4643.000000,4643.000000,4643.00000,4643.000000,4643.000000,4643.000000
mean,748.561921,758.535964,19.559724,0.051512,685.886498,49890.812243,10.11189,12.499460,9710.642020,59.134207
std,446.811723,446.798279,36.590275,0.089822,1048.160529,21493.140778,86.66593,108.564524,15407.915922,120.041439
min,71.000000,81.000000,1.000000,0.005000,1.600000,2080.000000,-999.00000,1.000000,26.000000,5.600000
25%,416.000000,424.250000,5.400000,0.010000,130.900000,39230.000000,1.40000,3.000000,1340.000000,29.800000
50%,641.000000,651.000000,9.200000,0.027000,390.666667,48960.000000,4.40000,4.600000,3620.000000,38.200000
75%,991.000000,1001.000000,20.000000,0.060000,931.300000,58380.000000,17.40000,8.000000,10984.000000,52.600000
max,2201.000000,2211.000000,827.800000,1.878000,29793.800000,397000.000000,1939.40000,6767.000000,217600.000000,3455.000000


In [10]:
# next deal with the -999 in Mo
# fist find the smallest non-negative value (we'll assume that's the detection limit)
Mo_min = data_p1['Mo'].abs().min()
Mo_min

1.0

In [11]:
# replace the negative values with this
#data_p1['Mo'] = data_p1['Mo'].replace(['-999'], ['1'])
data_p1['Mo'] = data_p1['Mo'].where(data_p1['Mo'] >= Mo_min, Mo_min)

In [12]:
data_p1.describe()

,from,to,As,Au,Pb,Fe,Mo,Cu,S,Zn
count,4643.000000,4643.000000,3187.000000,4643.000000,4643.000000,4643.000000,4643.000000,4643.000000,4643.000000,4643.000000
mean,748.561921,758.535964,19.559724,0.051512,685.886498,49890.812243,15.927096,12.499460,9710.642020,59.134207
std,446.811723,446.798279,36.590275,0.089822,1048.160529,21493.140778,39.429726,108.564524,15407.915922,120.041439
min,71.000000,81.000000,1.000000,0.005000,1.600000,2080.000000,1.000000,1.000000,26.000000,5.600000
25%,416.000000,424.250000,5.400000,0.010000,130.900000,39230.000000,1.400000,3.000000,1340.000000,29.800000
50%,641.000000,651.000000,9.200000,0.027000,390.666667,48960.000000,4.400000,4.600000,3620.000000,38.200000
75%,991.000000,1001.000000,20.000000,0.060000,931.300000,58380.000000,17.400000,8.000000,10984.000000,52.600000
max,2201.000000,2211.000000,827.800000,1.878000,29793.800000,397000.000000,1939.400000,6767.000000,217600.000000,3455.000000


for now I'll assume that the maximum values are correct, but may also be worth chacking for outliers or unrealistic values

## split data into different categories

In [13]:
data_A = data_p1[data_p1['Class']=='A']
data_A.describe()

,from,to,As,Au,Pb,Fe,Mo,Cu,S,Zn
count,2787.00000,2787.00000,1759.000000,2787.000000,2787.000000,2787.000000,2787.000000,2787.000000,2787.000000,2787.000000
mean,800.66272,810.62517,11.367720,0.060148,855.740655,49670.115005,22.091124,15.922254,9592.876495,59.397595
std,401.41024,401.39049,13.561059,0.084648,776.046533,14010.221927,48.323489,139.403212,13784.828762,138.969867
min,211.00000,221.00000,1.000000,0.005000,3.400000,2080.000000,1.000000,1.000000,26.000000,5.600000
25%,511.00000,521.00000,4.800000,0.017000,295.900000,40570.000000,3.000000,3.000000,1240.000000,28.600000
50%,681.00000,691.00000,7.400000,0.036000,672.200000,50380.000000,9.800000,5.000000,3410.000000,35.800000
75%,1011.00000,1021.00000,12.200000,0.072000,1176.000000,59340.000000,26.500000,9.000000,11070.000000,48.200000
max,2001.00000,2011.00000,150.200000,1.510000,7545.400000,125420.000000,1939.400000,6767.000000,86480.000000,3455.000000


In [14]:
data_B = data_p1[data_p1['Class']=='B']
data_B.describe()

,from,to,As,Au,Pb,Fe,Mo,Cu,S,Zn
count,1118.000000,1118.000000,743.000000,1118.000000,1118.000000,1118.000000,1118.000000,1118.00000,1118.000000,1118.000000
mean,542.368515,552.363801,33.689489,0.029151,337.047342,46763.619727,5.003643,8.11313,9820.637555,58.999616
std,452.243790,452.244021,52.144519,0.080302,1166.443998,26301.980838,11.288982,16.54162,16936.967407,85.692130
min,81.000000,91.000000,1.200000,0.005000,1.700000,3580.000000,1.000000,1.00000,37.000000,7.000000
25%,241.000000,251.000000,8.200000,0.006000,49.400000,33435.000000,1.000000,3.20000,1462.500000,33.450000
50%,391.000000,401.000000,20.200000,0.012000,129.600000,43710.000000,2.000000,5.00000,4740.000000,43.400000
75%,641.000000,651.000000,37.800000,0.028000,265.800000,55025.000000,3.600000,7.60000,12555.000000,60.600000
max,2131.000000,2141.000000,827.800000,1.878000,29793.800000,331400.000000,136.000000,284.80000,217600.000000,1750.400000


From comparing the means and ranges of the two datasets: Pb, Mo and Cu will be the most useful elements

In [15]:
data_unk = data_p1[data_p1['Class']=='?']
data_unk.describe()

,from,to,As,Au,Pb,Fe,Mo,Cu,S,Zn
count,738.000000,738.000000,685.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000
mean,864.170732,874.156369,25.269680,0.052775,572.903629,55461.658357,9.197116,6.218417,9988.741627,58.343434
std,500.214581,500.213708,48.906404,0.113642,1508.173098,32757.391204,20.044253,15.707372,18494.783717,81.397567
min,71.000000,81.000000,1.400000,0.005000,1.600000,18510.000000,1.000000,1.000000,50.000000,12.800000
25%,421.000000,431.000000,6.000000,0.008000,82.850000,43700.000000,1.000000,2.200000,1520.000000,31.000000
50%,821.000000,831.000000,11.200000,0.020500,177.700000,50230.000000,1.400000,3.100000,3260.000000,41.000000
75%,1261.000000,1271.000000,25.800000,0.058000,557.000000,57765.000000,6.800000,5.000000,7915.000000,57.400000
max,2201.000000,2211.000000,667.600000,1.618000,19000.000000,397000.000000,222.600000,209.800000,166040.000000,1237.600000


## Using a random forest for prediction

In [16]:
## A has more training data than B, so randomly select a data sample of the same size as B

data_A_lim = data_A.sample(1118, random_state=4)
data_A_lim.describe()

,from,to,As,Au,Pb,Fe,Mo,Cu,S,Zn
count,1118.000000,1118.000000,721.000000,1118.000000,1118.00000,1118.000000,1118.000000,1118.000000,1118.000000,1118.000000
mean,814.711986,824.654696,11.762317,0.058758,853.07860,49828.400707,23.026456,13.717365,9783.986593,61.424703
std,410.048030,410.012144,13.468142,0.081565,786.77152,14425.582959,65.313210,52.434118,13833.652245,137.728008
min,211.000000,221.000000,1.000000,0.005000,4.10000,2080.000000,1.000000,1.000000,44.000000,7.400000
25%,511.000000,521.000000,4.800000,0.017000,293.15000,40545.000000,3.000000,3.000000,1214.000000,28.400000
50%,691.000000,701.000000,7.400000,0.037000,649.30000,50580.000000,9.800000,5.000000,3556.515151,35.600000
75%,1051.000000,1061.000000,13.000000,0.070750,1164.85000,59895.000000,26.800000,8.950000,12142.500000,48.500000
max,2001.000000,2011.000000,120.400000,1.510000,7502.00000,98940.000000,1939.400000,904.400000,78080.000000,2347.200000


In [17]:
#Testing and training split, 1000 points for training and the rest for testing
train_A = data_A_lim.sample(1000, random_state=4)
test_A = data_A_lim.loc[~data_A_lim.index.isin(train_A.index)]
train_B = data_B.sample(1000, random_state=4)
test_B = data_B.loc[~data_B.index.isin(train_B.index)]

In [18]:
#Format data
train_x = pd.concat((train_A[['Au', 'Pb', 'Fe','Mo','Cu','S','Zn']],train_B[['Au', 'Pb', 'Fe','Mo','Cu','S','Zn']]), axis=0)
test_x = pd.concat((test_A[['Au', 'Pb', 'Fe','Mo','Cu','S','Zn']],test_B[['Au', 'Pb', 'Fe','Mo','Cu','S','Zn']]), axis=0)
train_y = np.ones(2000)
train_y[1000:2000] = 2
test_y = np.ones(236)
test_y[118:236] = 2

In [19]:
#Scale data using a basic scaler - may need improving
scaler_x = StandardScaler()
train_x_scaled = scaler_x.fit_transform(train_x)
test_x_scaled = scaler_x.transform(test_x)

In [20]:
# Fit random forest on original data
feature_names = np.array(test_x.columns.tolist())
forest = RandomForestClassifier(random_state=0)
forest.fit(train_x, train_y)

RandomForestClassifier(random_state=0)

In [21]:
# Check the importance of the elements
importances = forest.feature_importances_
importances

array([0.11824818, 0.26739641, 0.13014769, 0.17584987, 0.07504522,
       0.12583634, 0.1074763 ])

In [22]:
#order the elements by importance
forest_importances = pd.Series(importances, index=feature_names)
forest_importances.sort_values(ascending=False).head(7).index.tolist()

['Pb', 'Mo', 'Fe', 'S', 'Au', 'Zn', 'Cu']

In [23]:
# Fit random forest on scaled data
forest_s = RandomForestClassifier(random_state=0)
forest_s.fit(train_x_scaled, train_y)

RandomForestClassifier(random_state=0)

In [24]:
# Check the importance of the elements
importances_s = forest_s.feature_importances_
importances_s

array([0.11824818, 0.26739641, 0.13014769, 0.17584987, 0.07504522,
       0.12583634, 0.1074763 ])

In [25]:
#order the elements by importance
forest_importances_s = pd.Series(importances_s, index=feature_names)
forest_importances_s.sort_values(ascending=False).head(7).index.tolist()

['Pb', 'Mo', 'Fe', 'S', 'Au', 'Zn', 'Cu']

The scaling doesn't improve the results - dropping it

In [26]:
# Check model on test data
pred_y = forest.predict(test_x)
pred_y

array([2., 1., 1., 1., 1., 1., 2., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1.,
       1., 2., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 2.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 2., 2., 1., 2., 1., 1., 1., 2., 1., 1.,
       1., 1., 2., 2., 1., 1., 1., 2., 1., 1., 1., 2., 2., 1., 1., 2., 2.,
       2., 2., 2., 2., 2., 2., 1., 1., 2., 1., 2., 1., 1., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 2., 2., 1., 1., 1., 2., 1.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1.,
       2., 2., 2., 2., 1., 2., 2., 2., 2., 2., 2., 1., 2., 2., 2., 2., 2.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 1.

In [27]:
accuracy = accuracy_score(test_y, pred_y)
accuracy

0.8347457627118644

Basic random forest achieved a ~83% accuracy on the test data

## Predict class for unknown data

In [28]:
class_prediction = forest.predict(data_unk[['Au','Pb','Fe','Mo','Cu','S','Zn']])


## Summary
- Data was cleaned: As not used, Au and Mo detection limits converted, NaNs removed
- Class A was subsampled to have the same number of samples as Class B
- Each class was randomly split into 1000 training samples and 118 testing samples
- A random forest was trained on the data
- This found that the elements contributing the most were Pb and Mo (In decreasing order: ['Pb', 'Mo', 'S', 'Fe', 'Au', 'Zn', 'Cu'])
- Scaling the data did not improve the results
- An accuracy of ~ 83% was achieved on the test data
- This model was applied to the unlabelled data, see class_prediction
- A higher accuracy could potentially be acieved with a method such as neutral networks or Gaussian processes. This will depend on the noise in the data, but should be tested.
- Another option is to remove the least significant variables, however I was only using 7 elements and that's not a lot for random forests.